In [ ]:
!pip install sacremoses

In [ ]:
import warnings
import os
import re

import torch

from datasets import load_dataset, load_metricI

import pandas as pd

from transformers import AutoModel, AutoTokenizer, T5Tokenizer, BioGptTokenizer, BioGptForCausalLM, T5ForConditionalGeneration, AutoModelForSeq2SeqLM, BartTokenizer, BartForConditionalGeneration


In [ ]:
def generate_response(model, prompt_type, sent):

  incorrect_sentence1 = "what about your wit?"
  correct_sentence1 = "what about your weight?"

  incorrect_sentence2 = "my bible motion has been inconsistant"
  correct_sentence2 = "my bowel movement has been inconsistent"

  incorrect_sentence3 = "I had sensitis for the past couple off years"
  correct_sentence3 = "I have had sinusitis for the past couple of years"

  if prompt_type == "zero":
    prompt = f"""
          Correct the incorrect ASR transcription {sent} in the context of clinical conversations assuming incorect words may caused by similar sounding words:
          """
  if prompt_type == "one":
    prompt = f"""
          Correct the incorrect ASR transcription: {incorrect_sentence1} in the context of clinical conversations assuming incorect words may caused by similar sounding words: {correct_sentence1}

          Correct the incorrect ASR transcription: {sent} in the context of clinical conversations assuming incorect words may caused by similar sounding words:
          """
  if prompt_type == "two":
    prompt = f"""
          Correct the incorrect ASR transcription: {incorrect_sentence1} in the context of clinical conversations assuming incorect words may caused by similar sounding words: {correct_sentence1}

          Correct the incorrect ASR transcription: {incorrect_sentence2} in the context of clinical conversations assuming incorect words may caused by similar sounding words: {correct_sentence2}

          Correct the incorrect ASR transcription: {sent} in the context of clinical conversations assuming incorect words may caused by similar sounding words:
          """
  if prompt_type == "few":
    prompt = f"""
          Correct the incorrect ASR transcription: {incorrect_sentence1} in the context of clinical conversations assuming incorect words may caused by similar sounding words: {correct_sentence1}

          Correct the incorrect ASR transcription: {incorrect_sentence2} in the context of clinical conversations assuming incorect words may caused by similar sounding words: {correct_sentence2}

          Correct the incorrect ASR transcription: {incorrect_sentence3} in the context of clinical conversations assuming incorect words may caused by similar sounding words: {correct_sentence3}

          Correct the incorrect ASR transcription: {sent} in the context of clinical conversations assuming incorect words may caused by similar sounding words:
        """

  response = client.completions.create(
    model=model,
    prompt=prompt,
    temperature=1,
    max_tokens=256,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0
    )

  return response.choices[0].text

**Datasets**

In [ ]:
def gpt_inference_eval(dataset, model_, prompt_type):
  data  = load_dataset(dataset)
  df = gcd['test'].to_pandas()
  df['model_corrected'] = df['refs'].apply(lambda x: generate_response(model_, prompt_type, x))
  return df

In [ ]:
gcd  = load_dataset('gayanin/gcd-native-v8')
gcd_train = gcd['train'].to_pandas()
gcd_val= gcd['validation'].to_pandas()
gcd_test = gcd['test'].to_pandas()

In [ ]:
model = "gpt-3.5-turbo-instruct"
gcd_dataset = 'gayanin/gcd-native-v8'


In [ ]:
zero_gcd = gpt_inference_eval(gcd_dataset,model, "zero")
zero_gcd.to_csv('zero_gcd.csv')

In [ ]:
one_gcd = gpt_inference_eval(gcd_dataset,model, "one")
one_gcd.to_csv('one_gcd.csv')

In [ ]:
two_gcd = gpt_inference_eval(gcd_dataset,model, "two")
two_gcd.to_csv('two_gcd.csv')

In [ ]:
few_gcd = gpt_inference_eval(gcd_dataset,model, "few")
few_gcd.to_csv('few_gcd.csv')

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/biogpt")
model = BioGptForCausalLM.from_pretrained("microsoft/biogpt")

In [ ]:
inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")
outputs = model(**inputs, labels=inputs["input_ids"])
loss = outputs. v
logits = outputs.logits

In [7]:
outputs.loss

tensor(5.5997, grad_fn=<NllLossBackward0>)